In [ ]:
# Install missing libraries into this notebook’s environment

%pip install matplotlib folium seaborn scikit-learn xgboost

### **Objective for the Random Forest Model**

##### I am building a Random Forest model to predict the severity of accidents, so traffic authorities and emergency services can allocate resources more effectively and potentially reduce response times.
##### These actionable insights that can help in planning, resource allocation, and public safety measures. Helping Local traffic authorities, emergency responders, or city planners who need to anticipate high-severity accidents and prepare accordingly.

##### Data Cleaning & Preprocessing: Cleaned the dataset (final_cleaned_data.csv) by handling missing values, capping outliers, and correcting negatives.
##### EDA & Initial Feature Selection: Analyzed statistics, plots, and correlations to identify key features for predicting severity.
##### Deeper Feature Importance Analysis: Plan to apply permutation importance/SHAP values to confirm which features most influence accident severity.
##### Baseline Random Forest Model Training & Evaluation: Built an initial Random Forest model with a stratified split and evaluated it using classification metrics.
##### Hyperparameter Tuning & Cross-Validation: Will use GridSearchCV and K-fold cross-validation to optimize model parameters and robustness.
##### Additional Feature Engineering: Consider incorporating derived features (e.g., accident duration, time-based interactions) to further enhance predictions.
##### Integration & Deployment Considerations: Outline steps for future model deployment and monitoring to support operational use.
##### Overall Objective: Predict accident severity to help traffic authorities and emergency responders allocate resources effectively.

In [ ]:
# Importing Essential Libraries

import pandas as pd
import os
import numpy as ny
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import random
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier


##### **Load our CSV file into a DataFrame**

In [ ]:
accidents_df = pd.read_csv('10_US_Accidents_2016_2023.csv')

#### **Inspecting Data**

In [ ]:
# Number of Rows and Columns
print(accidents_df.shape)
# Inspecting the dataset structure
print(accidents_df.info())
# Inspect the dataset summary statistics
print(accidents_df.describe())

#### **Checking Severity Levels in dataset to base the predictive model on at a later stage**

In [ ]:
print("Unique severity values in dataset:")
print(accidents_df['Severity'].unique())

print("\nSeverity value counts in dataset:")
print(accidents_df['Severity'].value_counts())


## **Data Cleaning Steps**

#### **Removing Duplicates**

In [ ]:
# Removing duplicate rows 
accidents_df = accidents_df.drop_duplicates()
accidents_df

#### **Changing Format for columns**

In [ ]:
# I'm converting the Start_Time and End_Time columns to datetime using a known exact format.
accidents_df['Start_Time'] = pd.to_datetime(accidents_df['Start_Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
accidents_df['End_Time']   = pd.to_datetime(accidents_df['End_Time'],   format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Now, I want to see if any rows failed to parse and were set to NaT.
invalid_rows = accidents_df[accidents_df['Start_Time'].isna() | accidents_df['End_Time'].isna()]
num_invalid = len(invalid_rows)

if num_invalid > 0:
    print(f"{num_invalid} rows had invalid date/time formats and were set to NaT.")
else:
    print("All rows successfully converted to datetime without errors.")


In [ ]:
# Finally, I'll verify that both columns are now in datetime format (not object).

print(accidents_df.info())               # Shows overall DataFrame info, including dtypes
print(accidents_df['Start_Time'].dtype)  
print(accidents_df['End_Time'].dtype)  

#### **Checking & Attending for Negative values, if found**

In [ ]:
# Checking for negative values in the Distance(mi) column,
# which are likely data entry errors or invalid entries.
negative_distance_rows = accidents_df[accidents_df['Distance(mi)'] < 0]

if not negative_distance_rows.empty:
    print("Inconsistent values found (negative distances):")
    print(negative_distance_rows[['Distance(mi)', 'Start_Time', 'End_Time']])
    
    # Correct Approach Options:
    # 1) Drop these rows if they are deemed invalid:
    # accidents_df.drop(negative_distance_rows.index, inplace=True)
    
    # 2) Convert negative distances to positive if it's known they should be positive:
    # accidents_df.loc[accidents_df['Distance(mi)'] < 0, 'Distance(mi)'] = \
    #     accidents_df['Distance(mi)'].abs()
    
    # 3) Set them to NaN for later imputation:
    # accidents_df.loc[accidents_df['Distance(mi)'] < 0, 'Distance(mi)'] = float('nan')
else:
    print("No negative distances found.")

negative_distance_rows


#### **Standardize Text Fields and Clean Categorical Data**

In [ ]:
# Standardizing text fields to ensure consistency in analysis:
# 1) remove leading/trailing whitespace
# 2) converted all text to lowercase
text_columns = ['Street', 'City', 'County', 'State', 'Weather_Condition']

for col in text_columns:
    # Converting any non-string or missing values to string, then stripping and lowering.
    accidents_df[col] = accidents_df[col].astype(str).str.strip().str.lower()

accidents_df.head()

#### **Identify & Inspect Categorical Columns**

In [ ]:
# Defining which columns I consider 'categorical' for further review.
categorical_cols = [
    'Street', 'City', 'County', 'State', 'Weather_Condition',
    'Wind_Direction', 'Sunrise_Sunset', 'Civil_Twilight',
    'Nautical_Twilight', 'Astronomical_Twilight'
]

# Will look at the first few unique values in each categorical column
# to see what kind of data is available
for col in categorical_cols:
    unique_vals = accidents_df[col].unique()[:10]  # Show just the first 10 unique entries
    print(f"Unique values in '{col}' (showing up to 10): {unique_vals}")
    print("------------------------------------------------------------")


#### **Dropping Columns not needed for Analysis**

In [ ]:
# Dropping these columns because they will not contribute significantly to the analysis of accident severity (Eg: country only have USA data , ID not neeeded etc)
# I will revisit and potentially to drop more columns during the EDA phase as I gain further insights.
columns_to_drop = ['ID', 'Airport_Code','Country', 'Description'] 
accidents_df.drop(columns=columns_to_drop, inplace=True)

accidents_df.columns

#### **Identifying for outliers**

In [ ]:
# Approach: Will use the Interquartile Range (IQR) method. 
# Rows that fall outside [Q1 - 1.5IQR, Q3 + 1.5IQR] for any numeric column are considered outliers.

In [ ]:
# Visualizing each numeric column to inspect for outliers

numeric_cols = accidents_df.select_dtypes(include=[ny.number]).columns
print(numeric_cols)
for col in numeric_cols:
    accidents_df.boxplot(column=col)
    plt.title(f'Boxplot of {col}')
    plt.show()



#### **Capping the outliers** : Created new DataFrame 'accidents_df_capped' where I exclude the Severity column from outlier capping so that I don’t overwrite its values and lose other severity classes. Instead, all other numeric columns have been capped using the 1.5×IQR method.

In [ ]:
# Creating a new DataFrame named 'accidents_df_capped'
accidents_df_capped = accidents_df.copy()

# Identify numeric columns but exclude 'Severity' to preserve its distribution
numeric_cols = accidents_df_capped.select_dtypes(include=[ny.number]).columns
numeric_cols = [col for col in numeric_cols if col != 'Severity']

# Apply outlier capping using the 1.5 * IQR rule for each numeric column
for col in numeric_cols:
    Q1 = accidents_df_capped[col].quantile(0.25)
    Q3 = accidents_df_capped[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    accidents_df_capped[col] = ny.where(accidents_df_capped[col] < lower_bound, lower_bound, accidents_df_capped[col])
    accidents_df_capped[col] = ny.where(accidents_df_capped[col] > upper_bound, upper_bound, accidents_df_capped[col])

print("Data shape after capping outliers (excluding 'Severity'):", accidents_df_capped.shape)
print("Unique severity values in 'accidents_df_capped':", accidents_df_capped['Severity'].unique())
print("\nSeverity value counts in 'accidents_df_capped':")
print(accidents_df_capped['Severity'].value_counts())


#### **Checking for Missing Values**

In [ ]:
# I'm checking for missing values across all columns in accidents_df_capped.
missing_counts = accidents_df_capped.isnull().sum()

# Now will print only the columns that actually have missing values, along with their counts.
print("Columns with missing values and their counts:")
for col, count in missing_counts.items():
    if count > 0:
        print(f"{col}: {count}")


#### **Treating Missing Values**

In [ ]:
# Creating a new DataFrame for EDA after handling missing values
cleaned_df = accidents_df_capped.copy()

# Drop rows missing 'Start_Time' to ensure proper time-based forecasting
cleaned_df.dropna(subset=['Start_Time'], inplace=True)

# Dropped rows missing 'End_Time' to enable calculating accident duration if needed
cleaned_df.dropna(subset=['End_Time'], inplace=True)

# 'End_Lat' and 'End_Lng' have many missing values, and we already have 'Start_Lat'/'Start_Lng' for location, so I dropped them
cols_to_drop = ['End_Lat', 'End_Lng']
for col in cols_to_drop:
    if col in cleaned_df.columns:
        cleaned_df.drop(col, axis=1, inplace=True)

# For 'Precipitation(in)', assuming missing values mean no precipitation; filled them with 0
if 'Precipitation(in)' in cleaned_df.columns:
    cleaned_df['Precipitation(in)'].fillna(0, inplace=True)

# Impute numeric weather columns with their median
numeric_cols_median = [
    'Temperature(F)',
    'Wind_Chill(F)',
    'Humidity(%)',
    'Pressure(in)',
    'Visibility(mi)',
    'Wind_Speed(mph)'
]
for col in numeric_cols_median:
    if col in cleaned_df.columns:
        median_val = cleaned_df[col].median()
        cleaned_df[col].fillna(median_val, inplace=True)

# Impute categorical columns with their mode (most frequent value)
categorical_cols_mode = [
    'Zipcode',
    'Timezone',
    'Weather_Timestamp',
    'Wind_Direction',
    'Sunrise_Sunset',
    'Civil_Twilight',
    'Nautical_Twilight',
    'Astronomical_Twilight'
]
for col in categorical_cols_mode:
    if col in cleaned_df.columns:
        mode_val = cleaned_df[col].mode(dropna=True)
        if len(mode_val) > 0:
            cleaned_df[col].fillna(mode_val[0], inplace=True)

print("Shape of cleaned_df after handling missing values:", cleaned_df.shape)
print("\nRemaining missing values:")
print(cleaned_df.isnull().sum())

print("\nSeverity value counts in cleaned_df:")
print(cleaned_df['Severity'].value_counts())


## **EDA** 

#### Basic Summary Statistics, Distribution of Numerical Columns, Distribution of Categorical Columns, Correlation Analysis, Identify Relevant Features for Severity Prediction

#### **Basic Summary Statistics**: Quick review of min, max, mean, median, and standard deviation of numeric features to understand overall data spread.

In [ ]:
# Step 1: Basic Summary Statistics
# I'm printing basic statistics for numeric columns in our cleaned dataset.
# This helps us understand the data spread (min, max, mean, median, etc.) for each numeric feature.

print("Basic Summary Statistics for Numeric Columns:")
print(cleaned_df.describe())

# Optionally, to view overall DataFrame info (including data types and non-null counts), uncomment the following:
# print("\nDataFrame Information:")
# print(cleaned_df.info())



#### **Distribution of Numerical Columns** Visualizing each numeric feature in two ways (histograms and boxplots) to understand how values are spread out and spot any potential remaining anomalies.

In [ ]:

# Step 2: Distribution of Numerical Columns
# Plotting histograms and boxplots to see the spread of each numeric feature.

num_cols = cleaned_df.select_dtypes(include=['number']).columns

for col in num_cols:
    plt.figure(figsize=(8, 4))
    sns.histplot(data=cleaned_df, x=col, kde=True)
    plt.title(f'Histogram of {col}')
    plt.show()

    plt.figure(figsize=(8, 4))
    sns.boxplot(data=cleaned_df, x=col)
    plt.title(f'Boxplot of {col}')
    plt.show()


#### **Correlation Analysis** Generate a correlation matrix to identify which numeric features are most related to each other and to severity.

In [ ]:
# Step 3: Correlation Analysis for Numeric Columns
# A heatmap lets me see how strongly numeric features (including 'Severity') correlate.

# Focusing only on numeric columns for correlation
numeric_df = cleaned_df.select_dtypes(include=['int64','float64'])

plt.figure(figsize=(12, 10))
corr_matrix = numeric_df.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix for Numeric Features")
plt.show()

# Print correlation of each numeric feature with 'Severity' if it's in the numeric columns
if 'Severity' in numeric_df.columns:
    severity_corr = corr_matrix['Severity'].sort_values(ascending=False)
    print("Correlation of features with 'Severity':")
    print(severity_corr)


#### **Categorical Columns via Textual Summaries** how many unique categories each column has and prints the top five most frequent categories for a quick overview

In [ ]:
# Step 4 : Categorical Columns via Textual Summaries
cat_cols = cleaned_df.select_dtypes(include=['object']).columns

print("Textual Summary of Categorical Columns:\n")
for col in cat_cols:
    unique_vals = cleaned_df[col].nunique()
    print(f"{col}: {unique_vals} unique values")
    print("Top 5 Most Frequent Categories:")
    print(cleaned_df[col].value_counts().head(5))
    print("-"*50)


#### **Plot Only the Top N Categories** Instead of plotting all categories, it focuses on the top 10 most common categories in each column. This helps me visualize dominant categories

In [ ]:
# Step 5 : Plot Only the Top N Categories

cat_cols = cleaned_df.select_dtypes(include=['object']).columns

for col in cat_cols:
    # Get the top 10 categories
    top_10_categories = cleaned_df[col].value_counts().head(10).index
    plt.figure(figsize=(10, 4))
    sns.countplot(data=cleaned_df, x=col, order=top_10_categories)
    plt.title(f'Top 10 Categories for {col}')
    plt.xticks(rotation=45)
    plt.show()


#### **Zipcode Extraction**: The code creates a new column called zipcode_5 by taking the first five characters of the original Zipcode field. This allows to retain a simplified version of the zipcode for later hotspot or geographic analysis.
#### **Encode Before Plotting**: The code then label-encodes all other categorical columns. By removing zipcode_5 from the list of columns to be encoded, I ensured that the new column remains in its original string form for future use.

In [ ]:
# Step 5: Zipcode Handling and Label Encoding
# I am extracting the first 5 digits from Zipcode for future analysis, 
# while label-encoding the other categorical columns.

# Creating a new column 'zipcode_5' that stores the first 5 characters of Zipcode
cleaned_df['zipcode_5'] = cleaned_df['Zipcode'].astype(str).str[:5]

# Identify all categorical columns (object type)
cat_cols = cleaned_df.select_dtypes(include=['object']).columns.tolist()

# Remove 'zipcode_5' from cat_cols so it won't be label-encoded
if 'zipcode_5' in cat_cols:
    cat_cols.remove('zipcode_5')

from sklearn.preprocessing import LabelEncoder

# Label-encode each remaining categorical column
for col in cat_cols:
    encoder = LabelEncoder()
    cleaned_df[col] = encoder.fit_transform(cleaned_df[col])

print("Categorical columns have been label-encoded.")
print("A new column 'zipcode_5' has been created to preserve the first 5 digits for future analysis.")



####  Correlation Analysis after encoding categorical columns using only numeric features (including the previously categorical columns now encoded into numbers) and visualizing it as a heatmap

In [ ]:
# Re-running correlation analysis on updated DataFrame after encoding categorical columns
numeric_df_updated = cleaned_df.select_dtypes(include=['int64','float64'])

plt.figure(figsize=(15, 13))
corr_matrix_updated = numeric_df_updated.corr()
sns.heatmap(corr_matrix_updated, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix After Encoding")
plt.show()

if 'Severity' in numeric_df_updated.columns:
    severity_corr_updated = corr_matrix_updated['Severity'].sort_values(ascending=False)
    print("Correlation with 'Severity' after encoding:")
    print(severity_corr_updated)


#### **Identifying relevant features for severity prediction**


In [ ]:
# Keeping columns that are most likely useful based on domain knowledge (time, location, weather).
# Dropping columns with near-zero or redundant impact (like 'Wind_Chill(F)' as we already have 'Temperature(F)').

columns_to_keep = [
    'Severity',
    'Start_Time',        # needed for time-based features
    'Distance(mi)',
    'Temperature(F)',
    'Pressure(in)',
    'Humidity(%)',
    'Visibility(mi)',
    'Wind_Speed(mph)',
    'Weather_Condition', # already label-encoded
    'Sunrise_Sunset',    # label-encoded
    'Civil_Twilight',    # label-encoded
    'Nautical_Twilight', # label-encoded
    'Astronomical_Twilight', # label-encoded
    'City',              # label-encoded, might help for location-based patterns
    'Zipcode',           # label-encoded, optional if not too high-cardinality
    'Street',            # label-encoded, optional if not too high-cardinality
    'State'              # label-encoded
]

cleaned_df = cleaned_df[columns_to_keep].copy()
print("Remaining columns after selecting relevant features:", cleaned_df.columns.tolist())


#### **Feature Engineering & Final Cleaning**

In [ ]:
# Creating new columns from 'Start_Time' and dropping it afterward (if I do not need them directly)

# Convert Start_Time to datetime 
cleaned_df['Start_Time'] = pd.to_datetime(cleaned_df['Start_Time'], errors='coerce')

# Extracting time-based features
cleaned_df['Hour'] = cleaned_df['Start_Time'].dt.hour
cleaned_df['DayOfWeek'] = cleaned_df['Start_Time'].dt.dayofweek
cleaned_df['Month'] = cleaned_df['Start_Time'].dt.month

# Drop Start_Time as it's no longer needed
cleaned_df.drop(['Start_Time'], axis=1, inplace=True)

print("Shape after feature engineering & final cleaning:", cleaned_df.shape)
print("Current columns:", cleaned_df.columns.tolist())

## **Proceeding to modeling**

#### **Random Forest to predict accident severity**: Evaluated performance using classification report and a confusion matrix to see how well the model distinguishes different severity levels.

#### **Classification Report shows** : The results show that it does an excellent job on the most frequent class (severity 2), but struggles with the less frequent severities (especially severity 4). 

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
This is a common challenge in imbalanced datasets, where the largest class tends to dominate. The classification report shows that the model performs very well on severity 2 accidents—with high precision (0.89) and recall (0.96)—but it struggles with the less frequent classes. For severity 1, the model is correct 75% of the time when predicted, but only captures 22% of actual cases. Similarly, severity 3 is moderately predicted (precision 0.81, recall 0.63), while severity 4 predictions are weak (precision 0.58, recall 0.26). Overall, the model achieves an accuracy of 88% and a weighted F1 score of 0.86, largely driven by the strong performance on severity 2.

In [ ]:

# Splitting the data, training a Random Forest, and evaluating performance.

# Separate features (X) and target (y)
X = cleaned_df.drop('Severity', axis=1)
y = cleaned_df['Severity']


In [ ]:

# Creating train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42, 
                                                    stratify=y)  # stratify to keep class balance



In [ ]:
# Training Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)


In [ ]:
rf_model.fit(X_train, y_train)

In [ ]:
# Predicting on test data
y_pred = rf_model.predict(X_test)

# Evaluate
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

#### **Memory error for above code so will proceed with samples of data**

#### **Creating a new CSV with Cleaned DataFrame** : Will run Time Series model in a seperate file


In [ ]:
# Exporting the cleaned DataFrame to CSV
cleaned_df.to_csv("final_cleaned_data.csv", index=False)
print("CSV file 'final_cleaned_data.csv' has been created successfully.")


#### **Reducing the dataset size for a quick test & Feature Importance Visualization**: I am sampling a smaller subset (e.g., 20,000 rows) to ensure faster runtime and lower memory usage. Also, will plot which features are most influential in predicting accident severity.

In [ ]:
# Reduce sample size to 20,000 rows
sample_size = 20000

# Sample a subset of X and align y accordingly
X_small = X.sample(n=sample_size, random_state=42)
y_small = y.loc[X_small.index]

# Split the smaller subset into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small, test_size=0.2, random_state=42, stratify=y_small
)

print(f"Sampled dataset size: {X_small.shape}, train size: {X_train.shape}, test size: {X_test.shape}")

# Train a quick Random Forest model on the sampled data
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict on the test data
y_pred = rf_model.predict(X_test)

# Evaluate the model's performance
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


###############################################################################
# Feature Importance Visualization
# I will now plot which features are most influential in predicting accident severity.
###############################################################################
importances = rf_model.feature_importances_
feature_names = X_train.columns

feat_imp_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(data=feat_imp_df, x='Importance', y='Feature')
plt.title("Feature Importances (Sampled Dataset)")
plt.tight_layout()
plt.show()


##### **Brief Explanation of the Results**: I trained on 16,000 rows and tested on 4,000, which is a subset of the original dataset. This reduced memory usage and provided a quick check of the model’s feasibility.

##### **Classification Report**: Precision, Recall, F1 are shown for each severity class (1, 2, 3, 4).The model predicts severity 2 very well—its precision and recall are very high, meaning it rarely makes mistakes and catches almost all severity 2 accidents. 
##### In contrast, it completely misses severity 1 and 4 (both precision and recall are 0) and only partly identifies severity 3. 
##### Overall accuracy is 83%, but that mainly reflects the performance on the dominant severity 2 class.

##### **Feature Importances**: Street and Zipcode appear as the most influential features in this sample, followed by Distance(mi), City, Pressure(in), and Temperature(F). Time-based columns like Month and DayOfWeek have moderate importance, while Visibility(mi) seems less impactful here.

## **Hyperparameter Tuning with GridSearchCV**

#### This code tests different Random Forest parameters (n_estimators, max_depth, and min_samples_split) on a smaller data sample to quickly find the best setup.
#### 20,000 row sample to quickly identify the best hyperparameters. It then evaluates the best model on the test set using the F1 macro score.


In [ ]:
# Use a smaller sample for faster tuning
sample_size = 20000
X_small = X.sample(n=sample_size, random_state=42)
y_small = y.loc[X_small.index]

# Split the sampled data into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small, test_size=0.2, random_state=42, stratify=y_small
)

# Define a grid of hyperparameters to test
param_grid = {
    'n_estimators': [50, 100],  # number of trees
    'max_depth': [None, 10, 20],  # maximum depth of trees
    'min_samples_split': [2, 5]   # minimum samples to split a node
}

# Initialize a Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Set up GridSearchCV with 3-fold cross-validation
grid_search = GridSearchCV(estimator=rf,          # The base Random Forest model we want to tune
                           param_grid=param_grid, # The hyperparameter combinations to test
                           cv=3,                # Use 3-fold cross-validation for each combination
                           scoring='f1_macro',  # Evaluate each model with the macro F1 score
                             n_jobs=-1,         # Use all available CPU cores to speed up the process
                             verbose=1          # Print messages about the progress of the search
                             )

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Macro Score:", grid_search.best_score_)

# Evaluate the best model on the test set
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)
print("Classification Report on Test Data:")
print(classification_report(y_test, y_pred))


#### **Result of Hyperparameter Tuning with GridSearchCV** shows that the best parameters and F1 macro score indicate how well the model balances precision and recall across all severity classes in the sampled dataset. 
#### The classification report on the test data shows that the model performs strongly on severity 2 but still struggles with minority classes (severity 1 and 4). This is typical in imbalanced datasets, where the model focuses on the dominant class.

## **Cross-Validation**

#### **Why Cross-Validation?** It splits the data into multiple folds, training on several subsets and testing on the remaining fold each time. This provides a more stable measure of model performance compared to a single train/test split.
#### I have used **Fewer Rows** (10k): Less data in memory, so the model training for each fold is lighter. **3-Fold Cross-Validation**: Reduces how many parallel models need to fit. **n_jobs=1**: Runs on a single core, avoiding large spikes in memory from parallel processes.


In [ ]:
# Lower sample size to 10,000 rows
X_cv = X.sample(n=10000, random_state=42)
y_cv = y.loc[X_cv.index]

# Reinitialize the best model from the grid search
best_rf = grid_search.best_estimator_

# Use 3-fold CV and n_jobs=1 to reduce memory demands
cv_scores = cross_val_score(
    best_rf, X_cv, y_cv,
    cv=3, scoring='f1_macro', n_jobs=1
)

print("Cross-validation F1 Macro scores:", cv_scores)
print("Mean F1 Macro score:", ny.mean(cv_scores))
print("Std of F1 Macro scores:", ny.std(cv_scores))


#### **Results of Cross-Validation Results** above show F1 Macro scores from the 3-fold cross-validation are around 0.346 each time, with a very small standard deviation (about 0.0008). 
#### This means the model’s performance on the sampled data is stable but relatively low overall, indicating it consistently struggles with the less frequent severity classes.

## **Handling Class Imbalance with Weighted Random Forest**:

#### **Why Class Weights?** In an imbalanced dataset, the model often ignores minority classes. By assigning class_weight="balanced", each class influences the model more equally, potentially improving recall for severity 1 and 4.
#### **Sampled Data**: I still use a subset (10,000 rows) to avoid memory issues and keep training time short.
#### **Expected Outcome**: This approach may boost the model’s detection of less frequent severities, though it might slightly reduce overall accuracy on the majority class.

In [ ]:
# Use a smaller sample for speed
sample_size = 10000
X_imb = X.sample(n=sample_size, random_state=42)
y_imb = y.loc[X_imb.index]

# Split the sampled data with stratification
X_train_imb, X_test_imb, y_train_imb, y_test_imb = train_test_split(
    X_imb, y_imb, test_size=0.2, random_state=42, stratify=y_imb
)

# Assign class_weight="balanced" so minority classes get higher weighting
imb_rf_model = RandomForestClassifier(
    n_estimators=100, 
    random_state=42,
    class_weight="balanced"
)

imb_rf_model.fit(X_train_imb, y_train_imb)
y_pred_imb = imb_rf_model.predict(X_test_imb)

print("Classification Report with Class Weights:")
print(classification_report(y_test_imb, y_pred_imb))
print("\nConfusion Matrix with Class Weights:")
print(confusion_matrix(y_test_imb, y_pred_imb))


#### **Results of Class Weight**: By giving more weight to underrepresented classes, the model slightly improves recall for minority severities (like 1 and 4), but it may still struggle due to the heavy class imbalance. Overall accuracy might drop a bit on the majority class (severity 2), but I'm now catching at least some of the minority classes instead of missing them entirely.

### **Alternative Model (XGBoost) with Multi-Class Setup**

#### I'm using a smaller data sample again to compare XGBoost with our Random Forest approach. The goal is to see if XGBoost handles imbalanced classes better or yields higher performance.
#### I encode severity labels (1,2,3,4) to 0..3 so XGBoost can handle them properly.

In [ ]:
# Using a smaller sample for faster training
sample_size = 10000
X_xgb = X.sample(n=sample_size, random_state=42)
y_xgb = y.loc[X_xgb.index]

# Encode severity labels from {1,2,3,4} to {0,1,2,3}
# so XGBoost recognizes each class as an integer in [0..num_class-1].
y_encoder = LabelEncoder()
y_xgb_encoded = y_encoder.fit_transform(y_xgb)  # e.g., 1->0, 2->1, etc.

# Splitting the sampled data (with stratification)
X_train_xgb, X_test_xgb, y_train_xgb_enc, y_test_xgb_enc = train_test_split(
    X_xgb, y_xgb_encoded, test_size=0.2, random_state=42, 
    stratify=y_xgb_encoded
)

# Initialize an XGBoost classifier for multi-class (4 classes)
xgb_model = XGBClassifier(
    objective='multi:softmax',  # multi-class classification
    num_class=4,               # we have 4 severity classes
    n_estimators=100, 
    max_depth=6, 
    random_state=42,
    use_label_encoder=False,   # disable deprecation warning
    eval_metric='mlogloss'     # default multi-class metric
)

# Train the XGBoost model
xgb_model.fit(X_train_xgb, y_train_xgb_enc)

# Predict on the test set
y_pred_xgb_enc = xgb_model.predict(X_test_xgb)

# Decode predictions back to original labels (1..4)
y_pred_xgb = y_encoder.inverse_transform(y_pred_xgb_enc)

# Decode test labels too, for a proper comparison in the report
y_test_xgb = y_encoder.inverse_transform(y_test_xgb_enc)

# Evaluate the XGBoost model
print("Classification Report (XGBoost):")
print(classification_report(y_test_xgb, y_pred_xgb))

print("\nConfusion Matrix (XGBoost):")
print(confusion_matrix(y_test_xgb, y_pred_xgb))


#### **Results Explanation for XGBoost**: Model mostly predicts severity 2 and 4 well, but it completely misses severity 1 and only partly captures severity 3. Overall, while the model has an accuracy of 80%, its performance is poor for the less common classes, highlighting that the class imbalance is still a major issue.

## **Summary**

### **Initial Model**: The baseline Random Forest performed well for the dominant class (severity 2) but completely missed some minority classes, highlighting a common imbalance issue.

### **Feature Importance**: A quick Random Forest run showed which features drive the model’s predictions, confirming the importance of key weather and location variables.

### **Model Refinement:**: Hyperparameter tuning and cross-validation helped improve overall performance, although class imbalance still limits detection of less frequent severities.

### **Alternative Approach (XGBoost)**: XGBoost showed promise, with an overall accuracy of around 80%. However, while it improved predictions for severity 2 and 4, it still failed to detect severity 1 and only partially captured severity 3.



## **Conclusion & Next Steps**

#### Given the current results and time constraints, I am concluding the modeling phase here.

### **Limitations**:
#### (a) Persistent class imbalance, where minority classes (severity 1 and 3) remain poorly predicted. (b) Limited performance gains from hyperparameter tuning due to inherent data imbalances.

### **Future Improvements**:
### (a) Experiment with advanced resampling techniques (e.g., SMOTE) to handle class imbalance.(b) Test additional ensemble methods or boosting algorithms (or even deep learning models) to further enhance performance. (c) Incorporate more external or derived features (e.g., accident duration, detailed weather conditions) for richer predictions.

### This comprehensive approach—from data cleaning and EDA through feature importance analysis and alternative modeling—has provided valuable insights into predicting accident severity. Although my model performs well on the dominant class, future work will focus on improving detection for minority severities and exploring further model enhancements.